In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%matplotlib inline
from SparseFactorization.sparse_factorization import *
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import math
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
# Define roots of unity
def w_n(n):
    return np.e**((2 * np.pi * 1j) / n)

w_n(4)

(6.123233995736766e-17+1j)

In [39]:
# Omegas
def Omega_n(n):
    values = [w_n(i)**(-i) if i >= 1 else 1 for i in range(n)]
    return np.diag(values)

Omega_n(4)

array([[1.+0.00000000e+00j, 0.+0.00000000e+00j, 0.+0.00000000e+00j,
        0.+0.00000000e+00j],
       [0.+0.00000000e+00j, 1.+2.44929360e-16j, 0.+0.00000000e+00j,
        0.+0.00000000e+00j],
       [0.+0.00000000e+00j, 0.+0.00000000e+00j, 1.+2.44929360e-16j,
        0.+0.00000000e+00j],
       [0.+0.00000000e+00j, 0.+0.00000000e+00j, 0.+0.00000000e+00j,
        1.+6.10622664e-16j]])

In [40]:
# Define B_2 and B_4
def B_n(n):
    
    # Identities
    ident = np.eye(n//2)
    stacked_idnt = np.concatenate([ident, ident], axis=0)
    
    # Omegas
    o1 = Omega_n(n//2)
    o2 = -Omega_n(n//2)
    stacked_omeg = np.concatenate([o1, o2], axis=0)
    
    return np.concatenate([stacked_idnt, stacked_omeg], axis=1)

B_n(4)

array([[ 1.+0.0000000e+00j,  0.+0.0000000e+00j,  1.+0.0000000e+00j,
         0.+0.0000000e+00j],
       [ 0.+0.0000000e+00j,  1.+0.0000000e+00j,  0.+0.0000000e+00j,
         1.+2.4492936e-16j],
       [ 1.+0.0000000e+00j,  0.+0.0000000e+00j, -1.-0.0000000e+00j,
        -0.-0.0000000e+00j],
       [ 0.+0.0000000e+00j,  1.+0.0000000e+00j, -0.-0.0000000e+00j,
        -1.-2.4492936e-16j]])

In [41]:
# Define F_8 and F_4
def F_8():
    # First b8 factor
    B_8 = B_n(8)
    
    # Stack b4s
    B_4 = B_n(4)
    rows = []    
    for i in range(2):
        row = [np.zeros(B_4.shape) for j in range(i)] + [B_4] + [np.zeros(B_4.shape) for j in range(2-i-1)]
        rows.append(np.concatenate(row, axis=1))
    B_4_factor = np.concatenate(rows, axis=0)
    
    # Stack b2s
    B_2 = B_n(2)
    rows = []
    for i in range(4):
        row = [np.zeros(B_2.shape) for j in range(i)] + [B_2] + [np.zeros(B_2.shape) for j in range(4-i-1)]
        rows.append(np.concatenate(row, axis=1))
    B_2_factor = np.concatenate(rows, axis=0)
    
    # Permutation matrix
    perm = np.array([
        [1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0 ,0 ,0 ,0 ,0],
        [0, 0, 0, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1]
    ])
    
    return B_8.dot(B_4_factor).dot(B_2_factor).dot(perm)
    
F_8()

array([[ 1.+0.00000000e+00j,  1.+0.00000000e+00j,  1.+0.00000000e+00j,
         1.+0.00000000e+00j,  1.+0.00000000e+00j,  1.+0.00000000e+00j,
         1.+0.00000000e+00j,  1.+0.00000000e+00j],
       [ 1.+0.00000000e+00j,  1.+2.44929360e-16j,  1.+2.44929360e-16j,
         1.+4.89858720e-16j, -1.+0.00000000e+00j, -1.-2.44929360e-16j,
        -1.-2.44929360e-16j, -1.-4.89858720e-16j],
       [ 1.+0.00000000e+00j,  1.+2.44929360e-16j, -1.+0.00000000e+00j,
        -1.-2.44929360e-16j,  1.+0.00000000e+00j,  1.+2.44929360e-16j,
        -1.+0.00000000e+00j, -1.-2.44929360e-16j],
       [ 1.+0.00000000e+00j,  1.+6.10622664e-16j, -1.-2.44929360e-16j,
        -1.-8.55552023e-16j, -1.+0.00000000e+00j, -1.-6.10622664e-16j,
         1.+2.44929360e-16j,  1.+8.55552023e-16j],
       [ 1.+0.00000000e+00j, -1.+0.00000000e+00j,  1.+0.00000000e+00j,
        -1.+0.00000000e+00j,  1.+0.00000000e+00j, -1.+0.00000000e+00j,
         1.+0.00000000e+00j, -1.+0.00000000e+00j],
       [ 1.+0.00000000e+00j, -1.-2.